In [23]:
using Glob
using Dates
using DIVAnd
using NCDatasets
using GeoDatasets
using CairoMakie, GeoMakie
using JupyterFormatter
enable_autoformat()
include("./config.jl")

edit_mask!

In [15]:
topodir = "/home/ctroupin/data/GEBCO/"
topofile = joinpath(topodir, "gebco_30sec_16.nc")
bx, by, b = DIVAnd.extract_bath(topofile, true, lonr, latr)
b[b.<0.0] .= NaN;

lon_landsea, lat_landsea, landsea = GeoDatasets.landseamask(; resolution = 'i', grid = 5)
landsea[landsea.==2] .= 1;

In [34]:
xi, yi, mask = load_mask(topofile, true, lonr, latr, 0.0);
coordinatelist = read_polygon_json("./mask.json");
edit_mask!(xi, yi, mask, coordinatelist)

In [46]:
fig = Figure(size = (600, 600))
ga =
    GeoAxis(fig[1, 1], title = "GEBCO bathymetry", dest = "+proj=ortho +lon_0=15 +lat_0=35")
heatmap!(
    ga,
    lon_landsea,
    lat_landsea,
    landsea,
    colormap = Reverse(:greys),
    colorrange = [0, 2],
)
hm = heatmap!(ga, bx, by, b, colormap = :deep, colorrange = (0, 5000.0))
masklayer = contourf!(xi, yi, mask, levels = [0.0, 0.0001], colormap = Reverse(:binary))

xlims!(ga, -180, 180.0)
ylims!(ga, -90.0, 90.0)
hidedecorations!(ga)
Colorbar(fig[1, 2], hm, label = "m", labelrotation = 0)
save(joinpath(figdir, "gebco_bathy_mask.png"), fig)
fig

LoadError: SystemError: opening file "/home/ctroupin/Projects/EMODnet/EMODnet-Chemistry-GriddedMaps/figures/paper/gebco_bathy_mask.png": No such file or directory